In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
# Initialize an empty dictionary to store schema and table names
schema_tables = {}

# Loop through each schema folder in the specified path
for schema in dbutils.fs.ls('/mnt/bronze/'):
    schema_name = schema.name.split('/')[0]
    
    # Initialize a list to hold table names for the current schema
    tables = []
    
    # Loop through each table folder inside the current schema folder
    for table in dbutils.fs.ls(f'/mnt/bronze/{schema_name}'):
        table_name = table.name.split('/')[0]
        tables.append(table_name)
    
    # Add the schema and its tables to the dictionary
    schema_tables[schema_name] = tables

# Display the schema_tables dictionary
schema_tables

In [0]:
for schema, tables in schema_tables.items():
    for table in tables:
        path = f'/mnt/bronze/{schema}/{table}/{table}.parquet'
        df = spark.read.format('parquet').load(path)
        column = df.columns

        for col in column:
            if "date" in col or "Date" in col:
                df = df.withColumn(col, date_format(from_utc_timestamp(df[col].cast(TimestampType()), "UTC"),"yyyy-MM-dd"))

        df = df.dropDuplicates()

        output_path = f'/mnt/silver/{schema}/{table}/'
        df.write.format('delta').mode('overwrite').save(output_path)